In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from os.path import join

In [ ]:
from glofrim.glofrim_lib import configread
config = configread(r'../../environment.env')
root_dir = config.get('models', 'root_dir')

In [ ]:
from glofrim import PCR, WFL, CMF, DFM, LFP
# config_fn = join(root_dir, 'PCR_Elbe', 'setup_PCR_30min_Elbe.ini')
# bmi = PCR()

# config_fn = join(root_dir, 'WFL_Elbe', 'SBM', 'wflow_sbm.ini')
# bmi = WFL()

# config_fn = join(root_dir, 'CMF_Elbe', 'input_flood.temp')
# engine = config.get('engines', 'CMF')
# bmi = CMF(engine = engine)

# config_fn = join(root_dir, 'DFM_Elbe', 'Elbe_1way_1d2dFM_400m_3200m.mdu')
# engine = config.get('engines', 'DFM')
# bmi = DFM(engine = engine)

config_fn = join(root_dir, 'LFP_data', 'LFPmodel_Elbe.par')
engine = config.get('engines', 'LFP')
bmi = LFP(engine = engine)

test model control function

In [ ]:
bmi.initialize_config(config_fn)
bmi.get_start_time(), bmi.get_end_time()

In [ ]:
bmi.get_attribute_names()

In [ ]:
from datetime import datetime, timedelta
start_time = datetime(2000, 2, 1)
end_time = datetime(2000, 4, 1)
bmi.set_start_time(start_time)
bmi.set_end_time(end_time)
end_time == bmi.get_end_time(), start_time == bmi.get_start_time()


In [ ]:
bmi.initialize_model()

In [ ]:
# check start and end_time after initialization
print(bmi.get_end_time(), bmi.get_start_time())
assert end_time == bmi.get_end_time() and start_time == bmi.get_start_time()

In [ ]:
x, y = 10.1,53.4
bmi.get_grid()
if bmi.grid.type != 2:
    res = bmi.grid.res
    idx = bmi.grid.index(x, y)
    lfp_x, lfp_y = bmi.grid.xy(idx)
    print(idx)    

In [ ]:
if (bmi.grid._1d is not None) and (bmi.grid.type != 2):
    r, c = bmi.grid.index(bmi.grid._1d.nodes[:,0],bmi.grid._1d.nodes[:,1], flat_index=False)
    print(np.all(bmi.grid._inside(r, c)))

In [ ]:
try:
    bmi.update(1)
except ValueError, e:
    print(e)

In [ ]:
bmi.update(172800)

In [ ]:
bmi.get_current_time(), bmi._t, bmi._dt

In [ ]:
assert bmi._t == bmi.get_current_time()


In [ ]:
next_t = bmi.get_current_time() + timedelta(days=2)
bmi.update_until(next_t)
assert bmi._t == bmi.get_current_time()
print(bmi.get_current_time())

test model and attribute information functions

In [ ]:
assert isinstance(bmi.get_component_name(), str)
assert isinstance(bmi.get_input_var_names(), list)
assert isinstance(bmi.get_output_var_names(), list)

In [ ]:
bmi.get_var_type(bmi.get_input_var_names()[0])

In [ ]:
if hasattr(bmi, 'set_inpmat'):
    bmi.set_inpmat(pcr.get_grid_bounds(), pcr.get_grid_res())

In [ ]:
bmi.grid.type

In [ ]:
var_name = bmi.get_output_var_names()[0]
var_shape = bmi.get_var_shape(var_name)
bmi.set_value(var_name, np.ones(var_shape)) 
var_name, bmi.get_value(var_name).sum() == np.ones(var_shape).sum()

In [ ]:
if bmi.grid._1d is not None:
    inds = bmi.grid._1d.inds
    bmi.set_value_at_indices(var_name, inds, np.ones_like(inds)*78) 
    print(bmi.get_value_at_indices(var_name, inds).sum() ==np.ones_like(inds).sum()*78)

In [ ]:
bmi.finalize()